# Million Song Data Set & Complimentary Datasets Wrangling

**Import Libraries**

In [1]:
import pandas as pd
import numpy as np
import tables
import sys

sys.path.append('/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/MSongsDB/PythonSrc')
import hdf5_getters

**Upload MSD Data**

In [7]:
%%time 

example_hd5_path = '/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/MSD Complete Dataset/A/E/B/TRAEBAE128F9307121.h5'

h5_example = tables.open_file(example_hd5_path, mode='r')

#Fields potentially relevant.

num_songs_in_file = hdf5_getters.get_num_songs(h5_example) #single num

artist_name = hdf5_getters.get_artist_name(h5_example) # string

year = hdf5_getters.get_year(h5_example) # single num

tags = hdf5_getters.get_artist_mbtags(h5_example) #array

tags_freq = hdf5_getters.get_artist_mbtags(h5_example) #array

terms = hdf5_getters.get_artist_terms(h5_example) #array

terms_freq = hdf5_getters.get_artist_terms_freq(h5_example) #array

bars_start = hdf5_getters.get_bars_start(h5_example) #array

beats_start = hdf5_getters.get_beats_start(h5_example) #array

danceability = hdf5_getters.get_danceability(h5_example) #single num

duration = hdf5_getters.get_duration(h5_example) # single num in seconds

energy = hdf5_getters.get_energy(h5_example) # single num

key = hdf5_getters.get_key(h5_example) # single num

loudness = hdf5_getters.get_loudness(h5_example) # single num

mode = hdf5_getters.get_mode(h5_example) # single num

sections_start = hdf5_getters.get_sections_start(h5_example) #array

segments_loudness_max = hdf5_getters.get_segments_loudness_max(h5_example) #array

segments_loudness_max_time = hdf5_getters.get_segments_loudness_max_time(h5_example) #array

segments_loudness_start = hdf5_getters.get_segments_loudness_start(h5_example) #array

segments_pitches = hdf5_getters.get_segments_pitches(h5_example) #array

segments_start = hdf5_getters.get_segments_start(h5_example) #array

segments_timbre = hdf5_getters.get_segments_timbre(h5_example) #array

similar_artists = hdf5_getters.get_similar_artists(h5_example) #array

artist_hottness = hdf5_getters.get_artist_hotttnesss(h5_example) #single number

song_id = hdf5_getters.get_song_id(h5_example) #string

time_signature = hdf5_getters.get_time_signature(h5_example) #single number

title = hdf5_getters.get_title(h5_example) # string

track_id = hdf5_getters.get_track_id(h5_example) #string

release = hdf5_getters.get_release(h5_example) #string


CPU times: user 52.7 ms, sys: 2.44 ms, total: 55.2 ms
Wall time: 55.4 ms


In [3]:
mydict = {'num_songs_in_file': num_songs_in_file, 
          'artist_name': artist_name, 
          'year': year, 'tags': tags, 
          'tags_freq': tags_freq, 
          'terms': terms, 'terms_freq':terms_freq, 
          'beats_start': beats_start, 
          'danceability': danceability, 
          'duration': duration, 'energy': energy, 
          'key': key, 'loudness': loudness, 'mode': mode, 
          'sections_start': sections_start, 
          'segments_loudness_max': segments_loudness_max,
          'segments_loudness_max_time':segments_loudness_max_time, 
          'segments_loudness_start': segments_loudness_start,
          'segments_pitches':segments_pitches,
          'segments_start':segments_start,
          'segments_timbre':segments_timbre,
          'artist_hottness':artist_hottness,
          'song_id':song_id, 
          'time_signature':time_signature, 
          'title':title, 'track_id':track_id,'release':release}


print(mydict)


#Toughts & Ideas

    #Need to reduce/choose fewer features to extract?
    #Iterate over dircetory and create a list that contains rows. 
    #Iterate over directory and create a list that creates columns.
    #Iterate over directory and create a dictionary.
    #Iterate and puts files into an HDFS store, and then convert to what I need?
    #Once iterated, combine into dataframe.

{'num_songs_in_file': 1, 'artist_name': b'Mutandina', 'year': 2005, 'tags': array([], dtype='|S256'), 'tags_freq': array([], dtype='|S256'), 'terms': array([b'rock', b'salsa', b'latin', b'indie', b'creative commons',
       b'los angeles', b'funk', b'club', b'magnatune', b'jazz',
       b'latin alternative', b'mellow rock', b'dance', b'brazilian',
       b'pop', b'american', b'christian', b'argentina', b'latin funk',
       b'piano', b'90s', b'bass', b'percussion', b'drums'], dtype='|S256'), 'terms_freq': array([1.        , 0.59183251, 0.66864909, 0.83750833, 0.5134317 ,
       0.46909478, 0.59183251, 0.46909478, 0.36396262, 0.56636213,
       0.28498818, 0.28137394, 0.46909478, 0.28137394, 0.43363787,
       0.35317091, 0.25590356, 0.12273773, 0.06136886, 0.20386296,
       0.25590356, 0.09726735, 0.        , 0.        ]), 'beats_start': array([  1.81338,   2.3596 ,   2.89455,   3.5202 ,   4.12337,   4.69916,
         5.27918,   5.84793,   6.41527,   6.98402,   7.55841,   8.13279,
   

**Upload User Listening Data into a DataFrame**

In [4]:
%%time

#Uploading 10K rows for testing. 

users_path = '/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/Taste_Profile_Subset.txt'

users_listening_data = pd.read_csv(users_path, header = None, names = ['user_id','song_id','play_count'],
                                   nrows=10000,sep = '\s+')

users_listening_data.info()

print(users_listening_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
user_id       10000 non-null object
song_id       10000 non-null object
play_count    10000 non-null int64
dtypes: int64(1), object(2)
memory usage: 234.5+ KB
                                       user_id             song_id  play_count
0     b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995           1
1     b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAPDEY12A81C210A9           1
2     b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B           2
3     b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFNSP12AF72A0E22           1
4     b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFOVM12A58A7D494           1
5     b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBNZDC12A6D4FC103           1
6     b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBSUJE12A6D4F8CF5           2
7     b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBVFZR12A6D4F8AE3           1
8     b80344d063b5ccb32

**Upload Known Song Mismatches**

The purpose of this is to clean MSD and remove innacurate entries that can affect model fitting later on.

In [5]:
%%time 

msd_mismatch_path = '/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/MSD_Mismatches.csv'

msd_mismatch_data = pd.read_csv(msd_mismatch_path, names= ['error','song_id','track_id'], header=None)

msd_mismatch_data.info()

print(msd_mismatch_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19094 entries, 0 to 19093
Data columns (total 3 columns):
error       19094 non-null object
song_id     19094 non-null object
track_id    19094 non-null object
dtypes: object(3)
memory usage: 447.6+ KB
       error             song_id            track_id
0      ERROR  SOUMNSI12AB0182807  TRMMGKQ128F9325E10
1      ERROR  SOCMRBE12AB018C546  TRMMREB12903CEB1B1
2      ERROR  SOLPHZY12AC468ABA8  TRMMBOC12903CEB46E
3      ERROR  SONGHTM12A8C1374EF  TRMMITP128F425D8D0
4      ERROR  SONGXCA12A8C13E82E  TRMMAYZ128F429ECE6
5      ERROR  SOMBCRC12A67ADA435  TRMMNVU128EF343EED
6      ERROR  SOTDWDK12A8C13617B  TRMMNCZ128F426FF0E
7      ERROR  SOEBURP12AB018C2FB  TRMMPBS12903CE90E1
8      ERROR  SOSRJHS12A6D4FDAA3  TRMWMEL128F421DA68
9      ERROR  SOIYAAQ12A6D4F954A  TRMWHRI128F147EA8E
10     ERROR  SOUQUOG12A6D4F5965  TRMWCQW128F147DE05
11     ERROR  SORRXFX12A8C141030  TRMWRQM128F92CCC1C
12     ERROR  SOBWQMY12AB018AEAD  TRMWBQO128F935220C
13    

**Upload Known Song Duplicates**

The purpose of this is to clean MSD and remove known duplicates that can affect model fitting later on.

In [6]:
%%time

msd_duplicates_path = '/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/MSD_Duplicates.csv'

msd_duplicates_data = pd.read_csv(msd_duplicates_path, header=None, 
                                  names=['track_id','song_name','null'])

msd_duplicates_data[['song_name']] = msd_duplicates_data[['song_name']].fillna(method='ffill')

msd_duplicates_data = msd_duplicates_data[['track_id','song_name']].dropna().reset_index(drop=True)

msd_duplicates_data.info()

print(msd_duplicates_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131661 entries, 0 to 131660
Data columns (total 2 columns):
track_id     131661 non-null object
song_name    131661 non-null object
dtypes: object(2)
memory usage: 2.0+ MB
                  track_id                                          song_name
0       TRFCVSW12903D0A298               1 The Del Vikings - Whispering Bells
1       TRCWFEM128F9320F94               1 The Del Vikings - Whispering Bells
2       TRKYJRK12903CE6493               1 The Del Vikings - Whispering Bells
3       TRWTOBV128F9300F8A               1 The Del Vikings - Whispering Bells
4       TRWFIGX128F42920CA                  2 ANGELZOOM - Blasphemous rumours
5       TRFBNON128F4292174                  2 ANGELZOOM - Blasphemous rumours
6       TRXXZSH128F1468B07           3 Dion & The Belmonts - A Lover's Prayer
7       TRHCDQC12903CB39D3           3 Dion & The Belmonts - A Lover's Prayer
8       TRUYEKK128F42591B4                      4 Sentidos Opuestos - La Vid